# ЗАГРУЗКА

In [1]:
import pandas as pd
df_cop = pd.read_csv('tokenized_data.csv')

In [2]:
df_cop

,label,comment
0,toxic,скотина что сказать
1,normal,я сегодня проезжала по рабочей и между домами ...
2,normal,очередной лохотрон зачем придумывать очередной...
3,normal,ретро дежавю сложно понять чужое сердце лиш ощ...
4,normal,а когда мы статус агрогородка получили
...,...,...
386325,normal,спала в родительском доме на своей кровати про...
386326,normal,эх мы немного решили сократить путь сейчас уже...
386327,normal,что происходит со мной когда в эфире звучит мо...
386328,normal,любимая я подарю тебе эту звезду имя какой зве...


In [3]:
df_spam = pd.read_csv('spam_data_rus.csv')
df_spam

,comment,label
0,бесплатный вход в еженедельном соревновании дл...,spam
1,привет дорогая прошла уже неделя а ответа нет ...,spam
2,победитель как ценный клиент сети вы были выбр...,spam
3,если у вас есть месяц или более мобильного тел...,spam
4,шесть шансов выиграть наличные от до фунтов и ...,spam
...,...,...
1186,срочно розыгрыш прошлых выходных показывает чт...,spam
1187,срочно мы пытаемся связаться с вами розыгрыш н...,spam
1188,играйте в официальную игру на своем мобильном ...,spam
1189,спрос на мобильном если чатлинии включают бесп...,spam


In [4]:
df_cop.dropna(inplace=True)
df_cop.reset_index(inplace=True,drop=True)

In [5]:
df_cop['comment']

0                                       скотина что сказать
1         я сегодня проезжала по рабочей и между домами ...
2         очередной лохотрон зачем придумывать очередной...
3         ретро дежавю сложно понять чужое сердце лиш ощ...
4                    а когда мы статус агрогородка получили
                                ...                        
386310    спала в родительском доме на своей кровати про...
386311    эх мы немного решили сократить путь сейчас уже...
386312    что происходит со мной когда в эфире звучит мо...
386313    любимая я подарю тебе эту звезду имя какой зве...
386314    посмотри непытайтесьпокинутьомск сегодня в вав...
Name: comment, Length: 386315, dtype: object

In [6]:
import emoji
df_cop['comment'] = df_cop['comment'].apply(lambda s: emoji.replace_emoji(s, ''))

In [7]:
df_cop

,label,comment
0,toxic,скотина что сказать
1,normal,я сегодня проезжала по рабочей и между домами ...
2,normal,очередной лохотрон зачем придумывать очередной...
3,normal,ретро дежавю сложно понять чужое сердце лиш ощ...
4,normal,а когда мы статус агрогородка получили
...,...,...
386310,normal,спала в родительском доме на своей кровати про...
386311,normal,эх мы немного решили сократить путь сейчас уже...
386312,normal,что происходит со мной когда в эфире звучит мо...
386313,normal,любимая я подарю тебе эту звезду имя какой зве...


In [8]:
df_spam.isna().sum()

comment    2
label      0
dtype: int64

In [9]:
df_cop = pd.concat([df_cop,df_spam])
print(df_spam.isna().sum())
df_cop.dropna(inplace=True)

comment    2
label      0
dtype: int64


In [10]:
#df_cop.loc[df_cop['label']=='spam','label'] = 'toxic'

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df_cop['comment'])
X_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer().fit(X_counts)
X_tfidf = tf_transformer.transform(X_counts)
X_tfidf.shape

X = X_tfidf.copy()
y = df_cop['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  

In [20]:
from gensim.models import Word2Vec
X = df_cop['comment'].copy()
y = df_cop['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

model = w2v_model = Word2Vec(min_count=20,
                             window=2,
                             size=300,
                             sample=6e-5,
                             alpha=0.03,
                             min_alpha=0.0007,
                             negative=20,
                             workers=-1)

w2v_model.build_vocab(sentences, progress_per=10000)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

(95082909, 123979020)

In [27]:
X_train

[('крутит', 0.2795633375644684),
 ('псо', 0.27057307958602905),
 ('корсо', 0.2671842873096466),
 ('псж', 0.24746258556842804),
 ('корчит', 0.244126096367836),
 ('какие', 0.24325184524059296),
 ('выгонит', 0.24307788908481598),
 ('с', 0.23637565970420837),
 ('крутится', 0.23225802183151245),
 ('вывезти', 0.23000620305538177)]

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
params = {
    "loss" : ["hinge"],
    "alpha" : [0.1],
    "penalty" : ["l2"],
    'class_weight' : [None],
    'warm_start' : [True],
    'learning_rate' :['adaptive'],
    'eta0' : [0.001],
    'power_t' :[0.0001],
    'early_stopping' : [True],
    'validation_fraction' : [0.001]
    
}
 
model = SGDClassifier(max_iter=10000)
clf = GridSearchCV(model, param_grid=params)
clf.fit(X_train,y_train)
print(classification_report(y_test,clf.best_estimator_.predict(X_test)))

[('уау', 0.5035718083381653),
 ('щ', 0.4937123656272888),
 ('†', 0.48845529556274414),
 ('ў', 0.4673556387424469),
 ('бх', 0.4673093259334564),
 ('\u200bа', 0.4629170596599579),
 ('\u200bс', 0.4601266086101532),
 ('¡', 0.4565475583076477),
 ('зөв', 0.4467613697052002),
 ('нп', 0.445248007774353)]

In [31]:
df_cop[df_cop['label'] == 'spam']

ModuleNotFoundError: No module named 'fasttext'

In [29]:
df_cop

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for fasttext
Failed to build fasttext
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  C:\anaconda\Lib\site-packages\setuptools\dist.py:745: SetuptoolsDeprecationWarning: Invalid dash-separated options
  !!
  
          ********************************************************************************
          Usage of dash-separated 'description-file' will not be supported in future
          versions. Please use the underscore name 'description_file' instead.
  
          This deprecation is overdue, please update your project and remove deprecated
          calls to avoid build errors in the future.
  
          See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
          ********************************************************************************
  
  !!
    opt = self.warn_dash_deprecation(opt, section)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating bu

In [203]:
clf.best_params_

{'alpha': 0.1,
 'class_weight': None,
 'early_stopping': True,
 'eta0': 0.001,
 'learning_rate': 'adaptive',
 'loss': 'hinge',
 'penalty': 'l2',
 'power_t': 0.0001,
 'validation_fraction': 0.001,
 'warm_start': True}

In [ ]:
from imblearn.under_sampling import AllKNN
under =

In [ ]:
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(1000)
X_decom = clf.fit_transform(X_tfidf)
X_decom

In [12]:
X = X_decom.copy()
y = df_cop['label']

In [9]:
from imblearn.under_sampling import RandomUnderSampler
undersampler = RandomUnderSampler()
X_train, y_train = undersampler.fit_resample(X_train, y_train)

In [14]:
import numpy as np
from catboost import Pool, CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
train_pool = Pool(
    X_train, 
    y_train, 
    #feature_names=list(X_train)
)
valid_pool = Pool(
    X_test, 
    y_test,
    #feature_names=list(X_train)
)
catboost_params = {
    'iterations': 1000,
    'learning_rate': 0.01,
    'eval_metric': 'AUC',
    'task_type': 'GPU',
    'early_stopping_rounds': 20,
    #'use_best_model': True,
    #'verbose': 50
}
class_weight = compute_class_weight('balanced', classes=np.unique(y), y=y)
cat = CatBoostClassifier(**catboost_params,class_weights=class_weight)
cat.fit(X_train,y_train)
print(classification_report(y_test,cat.predict(X_test)))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 134ms	remaining: 2m 13s
1:	total: 209ms	remaining: 1m 44s
2:	total: 288ms	remaining: 1m 35s
3:	total: 365ms	remaining: 1m 30s
4:	total: 436ms	remaining: 1m 26s
5:	total: 502ms	remaining: 1m 23s
6:	total: 571ms	remaining: 1m 21s
7:	total: 642ms	remaining: 1m 19s
8:	total: 711ms	remaining: 1m 18s
9:	total: 780ms	remaining: 1m 17s
10:	total: 848ms	remaining: 1m 16s
11:	total: 911ms	remaining: 1m 15s
12:	total: 980ms	remaining: 1m 14s
13:	total: 1.04s	remaining: 1m 13s
14:	total: 1.11s	remaining: 1m 13s
15:	total: 1.18s	remaining: 1m 12s
16:	total: 1.25s	remaining: 1m 12s
17:	total: 1.31s	remaining: 1m 11s
18:	total: 1.38s	remaining: 1m 11s
19:	total: 1.45s	remaining: 1m 11s
20:	total: 1.51s	remaining: 1m 10s
21:	total: 1.58s	remaining: 1m 10s
22:	total: 1.65s	remaining: 1m 10s
23:	total: 1.72s	remaining: 1m 9s
24:	total: 1.79s	remaining: 1m 9s
25:	total: 1.85s	remaining: 1m 9s
26:	total: 1.92s	remaining: 1m 9s
27:	total: 1.99s	remaining: 1m 8s
28:	total: 2.06s	remaining: 1m 8s
2

242:	total: 14.6s	remaining: 45.5s
243:	total: 14.6s	remaining: 45.4s
244:	total: 14.7s	remaining: 45.3s
245:	total: 14.8s	remaining: 45.2s
246:	total: 14.8s	remaining: 45.2s
247:	total: 14.9s	remaining: 45.1s
248:	total: 14.9s	remaining: 45s
249:	total: 15s	remaining: 45s
250:	total: 15s	remaining: 44.9s
251:	total: 15.1s	remaining: 44.8s
252:	total: 15.2s	remaining: 44.7s
253:	total: 15.2s	remaining: 44.7s
254:	total: 15.3s	remaining: 44.6s
255:	total: 15.3s	remaining: 44.5s
256:	total: 15.4s	remaining: 44.5s
257:	total: 15.4s	remaining: 44.4s
258:	total: 15.5s	remaining: 44.3s
259:	total: 15.5s	remaining: 44.2s
260:	total: 15.6s	remaining: 44.2s
261:	total: 15.7s	remaining: 44.1s
262:	total: 15.7s	remaining: 44s
263:	total: 15.8s	remaining: 43.9s
264:	total: 15.8s	remaining: 43.9s
265:	total: 15.9s	remaining: 43.8s
266:	total: 15.9s	remaining: 43.7s
267:	total: 16s	remaining: 43.7s
268:	total: 16s	remaining: 43.6s
269:	total: 16.1s	remaining: 43.5s
270:	total: 16.2s	remaining: 43.5s

482:	total: 28s	remaining: 29.9s
483:	total: 28s	remaining: 29.9s
484:	total: 28.1s	remaining: 29.8s
485:	total: 28.1s	remaining: 29.8s
486:	total: 28.2s	remaining: 29.7s
487:	total: 28.2s	remaining: 29.6s
488:	total: 28.3s	remaining: 29.6s
489:	total: 28.3s	remaining: 29.5s
490:	total: 28.4s	remaining: 29.4s
491:	total: 28.5s	remaining: 29.4s
492:	total: 28.5s	remaining: 29.3s
493:	total: 28.6s	remaining: 29.3s
494:	total: 28.6s	remaining: 29.2s
495:	total: 28.7s	remaining: 29.2s
496:	total: 28.8s	remaining: 29.1s
497:	total: 28.8s	remaining: 29.1s
498:	total: 28.9s	remaining: 29s
499:	total: 28.9s	remaining: 28.9s
500:	total: 29s	remaining: 28.9s
501:	total: 29s	remaining: 28.8s
502:	total: 29.1s	remaining: 28.7s
503:	total: 29.1s	remaining: 28.7s
504:	total: 29.2s	remaining: 28.6s
505:	total: 29.2s	remaining: 28.6s
506:	total: 29.3s	remaining: 28.5s
507:	total: 29.4s	remaining: 28.4s
508:	total: 29.4s	remaining: 28.4s
509:	total: 29.5s	remaining: 28.3s
510:	total: 29.5s	remaining: 2

721:	total: 41.3s	remaining: 15.9s
722:	total: 41.3s	remaining: 15.8s
723:	total: 41.4s	remaining: 15.8s
724:	total: 41.4s	remaining: 15.7s
725:	total: 41.5s	remaining: 15.7s
726:	total: 41.5s	remaining: 15.6s
727:	total: 41.6s	remaining: 15.5s
728:	total: 41.6s	remaining: 15.5s
729:	total: 41.7s	remaining: 15.4s
730:	total: 41.7s	remaining: 15.4s
731:	total: 41.8s	remaining: 15.3s
732:	total: 41.8s	remaining: 15.2s
733:	total: 41.9s	remaining: 15.2s
734:	total: 42s	remaining: 15.1s
735:	total: 42s	remaining: 15.1s
736:	total: 42.1s	remaining: 15s
737:	total: 42.1s	remaining: 15s
738:	total: 42.2s	remaining: 14.9s
739:	total: 42.2s	remaining: 14.8s
740:	total: 42.3s	remaining: 14.8s
741:	total: 42.3s	remaining: 14.7s
742:	total: 42.4s	remaining: 14.7s
743:	total: 42.5s	remaining: 14.6s
744:	total: 42.5s	remaining: 14.6s
745:	total: 42.6s	remaining: 14.5s
746:	total: 42.6s	remaining: 14.4s
747:	total: 42.7s	remaining: 14.4s
748:	total: 42.7s	remaining: 14.3s
749:	total: 42.8s	remaining:

960:	total: 54.3s	remaining: 2.21s
961:	total: 54.4s	remaining: 2.15s
962:	total: 54.5s	remaining: 2.09s
963:	total: 54.5s	remaining: 2.04s
964:	total: 54.6s	remaining: 1.98s
965:	total: 54.6s	remaining: 1.92s
966:	total: 54.7s	remaining: 1.86s
967:	total: 54.7s	remaining: 1.81s
968:	total: 54.8s	remaining: 1.75s
969:	total: 54.8s	remaining: 1.7s
970:	total: 54.9s	remaining: 1.64s
971:	total: 54.9s	remaining: 1.58s
972:	total: 55s	remaining: 1.52s
973:	total: 55s	remaining: 1.47s
974:	total: 55.1s	remaining: 1.41s
975:	total: 55.1s	remaining: 1.35s
976:	total: 55.2s	remaining: 1.3s
977:	total: 55.2s	remaining: 1.24s
978:	total: 55.3s	remaining: 1.19s
979:	total: 55.3s	remaining: 1.13s
980:	total: 55.4s	remaining: 1.07s
981:	total: 55.4s	remaining: 1.01s
982:	total: 55.5s	remaining: 960ms
983:	total: 55.5s	remaining: 903ms
984:	total: 55.6s	remaining: 847ms
985:	total: 55.6s	remaining: 790ms
986:	total: 55.7s	remaining: 734ms
987:	total: 55.8s	remaining: 677ms
988:	total: 55.8s	remainin

In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test,cat.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.93      0.75      0.83    109687
       toxic       0.30      0.65      0.41     17802

    accuracy                           0.74    127489
   macro avg       0.62      0.70      0.62    127489
weighted avg       0.84      0.74      0.77    127489


In [10]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# Получение весов классов
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)

# Создание модели с взвешиванием классов
model = RandomForestClassifier()
model.fit(X_train,y_train)
print(classification_report(y_test,model.predict(X_test)))

NameError: name 'classification_report' is not defined

In [13]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42)

X_train, y_train = sm.fit_resample(X_train, y_train)

In [28]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN()

X_train, y_train = adasyn.fit_resample(X_train, y_train)

# Truncated

In [37]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import classification_report

clf2 = GaussianNB()
clf3 = BernoulliNB()

clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
print(classification_report(y_test,clf2.predict(X_test)),classification_report(y_test,clf3.predict(X_test)),sep = '\n')

              precision    recall  f1-score   support

      normal       0.90      0.50      0.65    109687
       toxic       0.18      0.67      0.28     17802

    accuracy                           0.53    127489
   macro avg       0.54      0.59      0.47    127489
weighted avg       0.80      0.53      0.60    127489

              precision    recall  f1-score   support

      normal       0.87      0.99      0.93    109687
       toxic       0.64      0.07      0.13     17802

    accuracy                           0.86    127489
   macro avg       0.75      0.53      0.53    127489
weighted avg       0.84      0.86      0.82    127489


# Sparced

In [23]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
model = SGDClassifier(loss='hinge')
model.fit(X_train,y_train)

SGDClassifier()

In [24]:
print(classification_report(y_test,model.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.95      0.95      0.95    109687
       toxic       0.71      0.71      0.71     17802

    accuracy                           0.92    127489
   macro avg       0.83      0.83      0.83    127489
weighted avg       0.92      0.92      0.92    127489


In [16]:
print(classification_report(y_test,model.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.90      1.00      0.95    109687
       toxic       0.97      0.31      0.47     17802

    accuracy                           0.90    127489
   macro avg       0.93      0.66      0.71    127489
weighted avg       0.91      0.90      0.88    127489


In [39]:
X_trainin.shape

(258841, 310760)

In [157]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
params = {
    "loss" : ["hinge"],
    "alpha" : [0.1],
    "penalty" : ["l2"],
    'class_weight' : [None],
    'warm_start' : [True],
    'learning_rate' :['adaptive'],
    'eta0' : [0.001],
    'power_t' :[0.0001],
    'early_stopping' : [True,False],
    'validation_fraction' : [0.001,0.0001]
    
}
 
model = SGDClassifier(max_iter=2000)
clf = GridSearchCV(model, param_grid=params)
clf.fit(X_train,y_train)

GridSearchCV(estimator=SGDClassifier(max_iter=2000),
             param_grid={'alpha': [0.1], 'class_weight': [None],
                         'early_stopping': [True, False], 'eta0': [0.001],
                         'learning_rate': ['adaptive'], 'loss': ['hinge'],
                         'penalty': ['l2'], 'power_t': [0.0001],
                         'validation_fraction': [0.001, 0.0001],
                         'warm_start': [True]})

In [53]:
clf.best_params_

{'alpha': 0.1,
 'class_weight': None,
 'early_stopping': True,
 'eta0': 0.001,
 'learning_rate': 'adaptive',
 'loss': 'hinge',
 'penalty': 'l2',
 'power_t': 0.0001,
 'validation_fraction': 0.0001,
 'warm_start': True}

In [26]:
clf.best_estimator_

SGDClassifier(alpha=0.1)

In [28]:
clf.best_params_

{'alpha': 0.1, 'loss': 'hinge', 'penalty': 'l2'}

In [158]:
print(classification_report(y_test,clf.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.96      0.97      0.97    109654
       toxic       0.82      0.73      0.77     17830

    accuracy                           0.94    127484
   macro avg       0.89      0.85      0.87    127484
weighted avg       0.94      0.94      0.94    127484


In [27]:
print(classification_report(y_test,clf.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.96      0.97      0.96    109687
       toxic       0.79      0.73      0.76     17802

    accuracy                           0.94    127489
   macro avg       0.87      0.85      0.86    127489
weighted avg       0.93      0.94      0.93    127489


In [10]:
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report
clf = NearestCentroid()

clf.fit(X_train, y_train)
print(classification_report(y_test,clf.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.95      0.97      0.96    109687
       toxic       0.81      0.71      0.76     17802

    accuracy                           0.94    127489
   macro avg       0.88      0.84      0.86    127489
weighted avg       0.93      0.94      0.93    127489


In [20]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
clf = BaggingClassifier(model)
param_grid = {
 'n_estimators': [5, 10, 15],
 'max_samples' : [0.6, 0.8, 1.0],
}
clf = GridSearchCV(clf, param_grid=param_grid)
clf.fit(X_train, y_train)


              precision    recall  f1-score   support

      normal       0.95      0.98      0.97    109687
       toxic       0.88      0.70      0.78     17802

    accuracy                           0.94    127489
   macro avg       0.92      0.84      0.87    127489
weighted avg       0.94      0.94      0.94    127489


In [21]:
print(classification_report(y_test,clf.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

      normal       0.95      0.98      0.97    109687
       toxic       0.88      0.70      0.78     17802

    accuracy                           0.94    127489
   macro avg       0.92      0.84      0.87    127489
weighted avg       0.94      0.94      0.94    127489


In [59]:
import pickle
with open("model.pickle",'wb') as file:
    pickle.dump(clf.best_estimator_,file)

In [196]:
import pickle
with open("model.pickle",'rb') as file:
     model = pickle.load(file)

In [197]:
model

SGDClassifier(alpha=0.1, early_stopping=True, eta0=0.001,
              learning_rate='adaptive', max_iter=2000, power_t=0.0001,
              validation_fraction=0.0001, warm_start=True)

In [159]:
df_cop

,label,comment
0,toxic,скотина что сказать
1,normal,я сегодня проезжала по рабочей и между домами ...
2,normal,очередной лохотрон зачем придумывать очередной...
3,normal,ретро дежавю сложно понять чужое сердце лиш ощ...
4,normal,а когда мы статус агрогородка получили
...,...,...
386325,normal,спала в родительском доме на своей кровати про...
386326,normal,эх мы немного решили сократить путь сейчас уже...
386327,normal,что происходит со мной когда в эфире звучит мо...
386328,normal,любимая я подарю тебе эту звезду имя какой зве...


In [163]:
df_cop = df_cop['comment'].apply(word_tokenize)
df_cop

0                                   [скотина, что, сказать]
1         [я, сегодня, проезжала, по, рабочей, и, между,...
2         [очередной, лохотрон, зачем, придумывать, очер...
3         [ретро, дежавю, сложно, понять, чужое, сердце,...
4             [а, когда, мы, статус, агрогородка, получили]
                                ...                        
386325    [спала, в, родительском, доме, на, своей, кров...
386326    [эх, мы, немного, решили, сократить, путь, сей...
386327    [что, происходит, со, мной, когда, в, эфире, з...
386328    [любимая, я, подарю, тебе, эту, звезду, имя, к...
386329    [посмотри, непытайтесьпокинутьомск, сегодня, в...
Name: comment, Length: 386315, dtype: object

In [181]:
from gensim.models import FastText
model = FastText()
model.build_vocab(corpus_iterable=df_cop)
model.train(corpus_iterable=df_cop, total_examples=df_cop.shape[0], epochs=30)

(118082315, 152902590)

In [191]:
model.wv.most_similar(input())

padla


[('эй', 0.571570873260498),
 ('хуец', 0.5626686215400696),
 ('клоун', 0.5573384761810303),
 ('гадёныш', 0.5466594696044922),
 ('пупсик', 0.5446894764900208),
 ('гавнюк', 0.5427184104919434),
 ('хун', 0.5411058664321899),
 ('ёб', 0.5351360440254211),
 ('пидар', 0.5311134457588196),
 ('нытик', 0.5182626843452454)]

In [195]:
model_text = model

In [74]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer
import re
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
stemmer = PorterStemmer()
def lemmatize(text):
    doc = re.sub(patterns, ' ', text)
    tokens = word_tokenize(doc, language='russian')
    lemmas = [morph.parse(token)[0].normal_form for token in tokens if token.lower() not in stopwords_ru]
    stems = [stemmer.stem(token) for token in tokens]
    return stems
def concat(text):
    text= ' '.join(text)
    return text
text = input()

text = [concat(lemmatize(text))]

X_counts = count_vect.transform(text)
X_counts.shape

X_tfidf = tf_transformer.transform(X_counts)
X_tfidf.shape

X = scaler.transform(X_tfidf)
clf.best_estimator_.predict(X)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


array(['toxic'], dtype='<U6')